# Very naive learning on the vector of surrounding bases with standardish machine learning algorithms from scikit sklearn

---

### Analysis

I will try to naively predict the mehtylation value (0,1) based on naive sequence context feauture vectors.

I will use a very popular "state of the art" general machine learning library: xgboost.
    



### Conlcusions

- It gains a slight, but real increse in precision over the other standard models (3-4%)

---

In [1]:
import numpy as np

import time
import xgboost as xgb

import sys
sys.path.append('../my_modules')
from loading_utils import read_my_data,create_sets

#workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
#subprocess.call(['mkdir',workdir])
#os.chdir(workdir)

Couldn't import dot_parser, loading of dot files will not be possible.


### Load data

In [2]:
x,y= read_my_data(fname='../prepare_data/small_meth_feat_vect.csv')

#select the whole dataset
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)= create_sets(
    x,y,N_train=80000,N_valid=10000,N_test=10000,length=2000)

Loading data... 


#### Create xgboost matrices

In [3]:
dtrain = xgb.DMatrix( train_x, label=train_y)
dvalid = xgb.DMatrix( valid_x, label=valid_y)
dtest = xgb.DMatrix( test_x, label=test_y)

#### Define params

In [7]:
param = {'bst:max_depth':7,
         'bst:eta':0.05,
         'min_child_weight':1,
         'colsample_bytree':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':4}

evallist  = [(dvalid,'eval'), (dtrain,'train')]

#### Fit

- The evaluation set is used to monitor the learning

In [8]:
#train
num_round = 300
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round)
                #early_stopping_rounds=5,
                #feval=feval) #not using my own evaluation function because its sloW!

train_bst_pred=bst.predict(dtrain)
test_bst_pred=bst.predict(dvalid)

[0]	eval-error:0.381500	train-error:0.377613
[1]	eval-error:0.371000	train-error:0.360112
[2]	eval-error:0.355500	train-error:0.335687
[3]	eval-error:0.331200	train-error:0.310950
[4]	eval-error:0.322200	train-error:0.293475
[5]	eval-error:0.315700	train-error:0.284800
[6]	eval-error:0.311100	train-error:0.274213
[7]	eval-error:0.305200	train-error:0.265013
[8]	eval-error:0.303100	train-error:0.260350
[9]	eval-error:0.296700	train-error:0.253650
[10]	eval-error:0.292700	train-error:0.246837
[11]	eval-error:0.288000	train-error:0.242575
[12]	eval-error:0.288900	train-error:0.238412
[13]	eval-error:0.284500	train-error:0.235300
[14]	eval-error:0.282700	train-error:0.231562
[15]	eval-error:0.277700	train-error:0.228600
[16]	eval-error:0.277800	train-error:0.224912
[17]	eval-error:0.277300	train-error:0.220588
[18]	eval-error:0.273700	train-error:0.217525
[19]	eval-error:0.273000	train-error:0.215163
[20]	eval-error:0.271300	train-error:0.212962
[21]	eval-error:0.271000	train-error:0.21126

#### Final scores

In [9]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.94145
validation score: 0.785
test score: 0.7864
